In [18]:
# Importing libraries
import pandas as pd
import numpy as np

In [19]:
# Importing dataset, creating dataframe df to view it
df = pd.read_csv('https://raw.githubusercontent.com/szilvasipeter2000/coding1_team-project/main/cardio_fitness.csv?token=GHSAT0AAAAAACHJH3V252GNRW5UO3L2QCLCZLXRFCQ')
df

,Product,Age,Gender,Education,MaritalStatus,Usage,Fitness,Income,Miles
0,TM195,18,Male,14,Single,3,4,29562,112
1,TM195,19,Male,15,Single,2,3,31836,75
2,TM195,19,Female,14,Partnered,4,3,30699,66
3,TM195,19,Male,12,Single,3,3,32973,85
4,TM195,20,Male,13,Partnered,4,2,35247,47
...,...,...,...,...,...,...,...,...,...
175,TM798,40,Male,21,Single,6,5,83416,200
176,TM798,42,Male,18,Single,5,4,89641,200
177,TM798,45,Male,16,Single,5,5,90886,160
178,TM798,47,Male,18,Partnered,4,5,104581,120


In [22]:
# Showing comparative descriptive statistics for each product comparatively
df.groupby('Product').describe().round(2)

Age                                              Education         \
        count   mean   std   min    25%   50%    75%   max     count   mean   
Product                                                                       
TM195    80.0  28.55  7.22  18.0  23.00  26.0  33.00  50.0      80.0  15.04   
TM498    60.0  28.90  6.65  19.0  24.00  26.0  33.25  48.0      60.0  15.12   
TM798    40.0  29.10  6.97  22.0  24.75  27.0  30.25  48.0      40.0  17.32   

         ...   Income           Miles                                     \
         ...      75%       max count    mean    std   min    25%    50%   
Product  ...                                                               
TM195    ...  53439.0   68220.0  80.0   82.79  28.87  38.0   66.0   85.0   
TM498    ...  53439.0   67083.0  60.0   87.93  33.26  21.0   64.0   85.0   
TM798    ...  90886.0  104581.0  40.0  166.90  60.07  80.0  120.0  160.0   

                       
           75%    max  
Product                
TM195     94.0  188.0  
TM498    106.0  212.0  
TM798    200.0  360.0  

[3 rows x 48 columns]

In [23]:
# Importing libraries for regressions
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mizani import transforms
from stargazer.stargazer import Stargazer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss

warnings.filterwarnings("ignore")

In [24]:
# Creating dummy variables for TM498 and TM798
df['ID_TM498'] = (df['Product'] == 'TM498').astype(int)
df['ID_TM798'] = (df['Product'] == 'TM798').astype(int)

In [26]:
# Creating a lnw variable for ln income
df['lnw'] = np.log(df["Income"])

In [27]:
# Creating dummy variables for gender (female=1) and fitness level >= 4 as 'high_fitness'
df['female'] = (df['Gender'] == 'Female').astype(int)
df['high_fitness'] = (df['Fitness'] >= 4).astype(int)

In [38]:
# Creating multiple regression models
reg1 = smf.ols("lnw ~ ID_TM498 + ID_TM798", data=df).fit(cov_type="HC1") # ln inc on all products
reg2 = smf.ols("lnw ~ ID_TM498 + ID_TM798 + female", data=df).fit(cov_type='HC1') # Adding gender
reg3 = smf.ols('lnw ~ ID_TM498 + ID_TM798 + female + Age + high_fitness', data=df).fit(cov_type='HC1') # Adding age and high_fitness

In [39]:
# Stargazer to view regression results
stargazer = Stargazer([reg1, reg2, reg3])
stargazer.covariate_order(['ID_TM498', 'ID_TM798', 'female', "Age", "high_fitness", "Intercept"])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer

Results from these regression analysis show that product TM497 users have on average 5.7% more income than TM195 users when not controling for anything else.

TM798 users are expected to have 47.4% more income than TM195 users.

When controlling for gender, female users of TM195 have .5% less income on average.

The coefficient on age shows that TM195 users, when accounting for each additional year of age, have 2.2% more inc with each year of age.

For high_fitness, those considered to be fit have 1.4% less income. This could imply that TM195 users are less fit and more fit users use the other products.